# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
import nltk
from random import randint

from sqlalchemy import create_engine

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
try:
    from textaugment import Wordnet
except:
    !pip install textaugment
    from textaugment import Wordnet

import warnings

warnings.simplefilter('ignore')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
    100% |████████████████████████████████| 645kB 7.1MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 5.7MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 3.2MB/s eta 0:00:01
    100% |████████████████████████████████| 1.0MB 6.5MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 5.2MB/s ta 0:00:011    78% |█████████████████████████▏      | 51kB 11.7MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 7.8MB/s ta 0:00:011
    100% |██████████████████████

In [21]:
def show_model_dif(y_test, y_pred1, y_pred2, y_test2='NA'):
    
    check = True
    try:
        if y_test2 == "NA":
            pass
    except:
        check = False
    
    if check:
        
        accuracy, precision, recall, f1 = get_mean_score(np.array(y_test), y_pred1, Y.columns)
        tuned_accuracy, tuned_precision, tuned_recall, tuned_f1 = get_mean_score(np.array(y_test), y_pred2, Y.columns)

        print(f" {f'ACCURACY':^20} | model 1: {round(accuracy, 4)} | model 2: {round(tuned_accuracy, 4)} -> dif: {round(tuned_accuracy - accuracy, 4):>7}")
        print(f" {f'PRECISION':^20} | model 1: {round(precision, 4)} | model 2: {round(tuned_precision, 4)} -> dif: {round(tuned_precision - precision, 4):>7}")
        print(f" {f'RECALL':^20} | model 1: {round(recall, 4)} | model 2: {round(tuned_recall, 4)} -> dif: {round(tuned_recall - recall, 4):>7}")
        print(f" {f'F1':^20} | model 1: {round(f1, 4)} | model 2: {round(tuned_f1, 4)} -> dif: {round(tuned_f1 - f1, 4):>7}")
           
    else:
        accuracy, precision, recall, f1 = get_mean_score(np.array(y_test), y_pred1, Y.columns)
        tuned_accuracy, tuned_precision, tuned_recall, tuned_f1 = get_mean_score(np.array(y_test2), y_pred2, Y.columns)

        print(f" {f'ACCURACY':^20} | model 1: {round(accuracy, 4)} | model 2: {round(tuned_accuracy, 4)} -> dif: {round(tuned_accuracy - accuracy, 4):>7}")
        print(f" {f'PRECISION':^20} | model 1: {round(precision, 4)} | model 2: {round(tuned_precision, 4)} -> dif: {round(tuned_precision - precision, 4):>7}")
        print(f" {f'RECALL':^20} | model 1: {round(recall, 4)} | model 2: {round(tuned_recall, 4)} -> dif: {round(tuned_recall - recall, 4):>7}")
        print(f" {f'F1':^20} | model 1: {round(f1, 4)} | model 2: {round(tuned_f1, 4)} -> dif: {round(tuned_f1 - f1, 4):>7}")
        

In [2]:
# load data from database
engine = create_engine('sqlite:///teste.db')
df = pd.read_sql_table('message', con=engine)

X = df['message']
Y = df.drop(columns=['id','message','original','genre'], axis=1)
for column in Y.columns:
    Y[column] = pd.to_numeric(Y[column])


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    lemmatizer = WordNetLemmatizer()
    
    tokens = [lemmatizer.lemmatize(word) for word in word_tokenize(text) if word not in stop_words]
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# Create pipeline with Classifier
moc = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])

stop_words= stopwords.words('english')

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

pipeline.fit(X_train.as_matrix(), y_train.as_matrix())

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [6]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
for i, col in enumerate(Y.columns): 
    print("_"*80)
    print('\n')
    print(f'{col.upper():^70}\n')
    print(classification_report(list(y_test.values[:, i]), list(y_pred[:, i])))

________________________________________________________________________________


                               RELATED                                

             precision    recall  f1-score   support

          0       0.63      0.46      0.53      1553
          1       0.84      0.92      0.88      5001

avg / total       0.79      0.81      0.80      6554

________________________________________________________________________________


                               REQUEST                                

             precision    recall  f1-score   support

          0       0.90      0.98      0.93      5461
          1       0.80      0.43      0.56      1093

avg / total       0.88      0.89      0.87      6554

________________________________________________________________________________


                                OFFER                                 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      652


________________________________________________________________________________


                         OTHER_INFRASTRUCTURE                         

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6277
          1       0.20      0.00      0.01       277

avg / total       0.93      0.96      0.94      6554

________________________________________________________________________________


                           WEATHER_RELATED                            

             precision    recall  f1-score   support

          0       0.87      0.95      0.91      4738
          1       0.83      0.64      0.72      1816

avg / total       0.86      0.86      0.86      6554

________________________________________________________________________________


                                FLOODS                                

             precision    recall  f1-score   support

          0       0.94      1.00      0.97      60

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f2f3ddcd950>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [9]:
# Define performance metric for use in grid search scoring object
def performance_metric(y_true, y_pred):
    """Calculate median F1 score for all of the output classifiers
    
    Args:
    y_true: array. Array containing actual labels.
    y_pred: array. Array containing predicted labels.
        
    Returns:
    score: float. Median F1 score for all of the output classifiers
    """
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score


def get_results(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args:
    actual: array. Array containing actual labels.
    predicted: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the predicted fields.
       
    Returns:
    metrics_df: dataframe. Dataframe containing the accuracy, precision, recall 
    and f1 score for a given set of actual and predicted labels.
    """
    columns = {}
    
    # Calculate evaluation metrics for each set of labels
    for i, col in enumerate(col_names):
        
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i])
        recall = recall_score(actual[:, i], predicted[:, i])
        f1 = f1_score(actual[:, i], predicted[:, i])
        
        columns[col] = {"accuracy": accuracy,"precision":precision,"recall":recall,"f1":f1}
      
    return columns

def get_mean_score(real, predicted, col_names):
    
    results = get_results(real, predicted, col_names)
    
    accuracy = []
    precision = []
    recall = []
    f1 = []

    for key in results.keys():
        accuracy.append(results[key]['accuracy'])
        precision.append(results[key]['precision'])
        recall.append(results[key]['recall'])
        f1.append(results[key]['f1'])
        
    accuracy = np.mean(accuracy)
    precision = np.mean(precision)
    recall = np.mean(recall)
    f1 = np.mean(f1)
    
    return accuracy, precision, recall, f1
        
    

In [10]:
parameters = {'clf__estimator__min_samples_leaf':[1, 2, 5, 10],
              'clf__estimator__min_samples_split': [2, 5, 10, 15, 100]}

In [11]:
scorer = make_scorer(performance_metric)

In [12]:
%%time
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(81)
tuned_model = cv.fit(X_train.values, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, score=0.11436049801924164, total=  48.3s
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   58.8s remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, score=0.09196753339580985, total=  48.4s
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, score=0.13263995891114536, total=  48.8s
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=5 


...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  8.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=15, score=0.1119362920544023, total=  36.9s
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=15 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=15, score=0.13313769563769565, total=  37.4s
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=15 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=15, score=0.1432010929109575, total=  37.4s
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=100 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=100, score=0.09717868338557992, total=  32.8s
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=100 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=100, score=0.09321543208437283, total=  32.5s
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=10

[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, score=0.0, total=  21.8s
[CV] clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10 
[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, score=0.0, total=  21.7s
[CV] clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10 
[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, score=0.0, total=  21.7s
[CV] clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=15 
[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=15, score=0.0, total=  21.8s
[CV] clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=15 
[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=15, score=0.0, total=  21.5s
[CV] clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=15 
[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 38.9min finished


CPU times: user 39min 50s, sys: 492 ms, total: 39min 50s
Wall time: 40min 1s


In [13]:
# Best mean test score
np.max(tuned_model.cv_results_['mean_test_score'])

0.15639273657495792

In [14]:
# Parameters for best mean test score
tuned_model.best_params_

{'clf__estimator__min_samples_leaf': 1, 'clf__estimator__min_samples_split': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# Calculate evaluation metrics for test set
tuned_y_pred = tuned_model.predict(X_test)

#### 7.1 Show mean results

In [17]:
accuracy, precision, recall, f1 = get_mean_score(np.array(y_test), y_pred, Y.columns)
tuned_accuracy, tuned_precision, tuned_recall, tuned_f1 = get_mean_score(np.array(y_test), tuned_y_pred, Y.columns)

print(f" {f'ACCURACY':^20} | default: {round(accuracy, 4)} tuned: {round(tuned_accuracy, 4)} dif: {round(tuned_accuracy - accuracy, 4):>7}")
print(f" {f'PRECISION':^20} | default: {round(precision, 4)} tuned: {round(tuned_precision, 4)} dif: {round(tuned_precision - precision, 4):>7}")
print(f" {f'RECALL':^20} | default: {round(recall, 4)} tuned: {round(tuned_recall, 4)} dif: {round(tuned_recall - recall, 4):>7}")
print(f" {f'F1':^20} | default: {round(f1, 4)} tuned: {round(tuned_f1, 4)} dif: {round(tuned_f1 - f1, 4):>7}")
#print(accuracy, precision, recall, f1)

       ACCURACY       | default: 0.9452 tuned: 0.9463 dif:  0.0011
      PRECISION       | default: 0.5462 tuned: 0.5365 dif: -0.0097
        RECALL        | default: 0.1855 tuned: 0.1999 dif:  0.0144
          F1          | default: 0.2417 tuned: 0.2515 dif:  0.0099


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 8.1 Changing Random Forest to Adaptive Boosting (AdaBoost)

In [18]:
# Create pipeline with Classifier
moc2 = MultiOutputClassifier(AdaBoostClassifier())

pipeline_ada = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc2)
    ])

In [19]:
pipeline_ada.fit(X_train.as_matrix(), y_train.as_matrix())

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [20]:
ada_y_pred = pipeline_ada.predict(X_test)

In [22]:
show_model_dif(y_test, tuned_y_pred, ada_y_pred)

       ACCURACY       | model 1: 0.9463 | model 2: 0.9478 -> dif:  0.0014
      PRECISION       | model 1: 0.5365 | model 2: 0.5598 -> dif:  0.0233
        RECALL        | model 1: 0.1999 | model 2: 0.3229 -> dif:   0.123
          F1          | model 1: 0.2515 | model 2: 0.3945 -> dif:   0.143


Here we see some improvement in performance using Adaboost.

### 8.2 Handling imbalanced dataset:

Functions for imbalanced dataset

In [23]:
def synonym_tokenize(text):
    """
    Takes a text and return tokenize synonyms 
    """
    t = Wordnet(v=True, n=True, p=0.7)
    return [t.augment(word) for word in tokenize(text)]


def oversample_nlp(dataframe):
    
    obj = dataframe.mean()
    obj = obj.sort_values(ascending=False)

    columns_perc = {}
    
    new_dataframe = pd.DataFrame(columns = dataframe.columns)

    for key,value in obj.iteritems():
        if key not in ['id','message','original','genre']:
            columns_perc[key] = round(value*100, 1)
   
    for column in dataframe.columns:

        if column not in ['id','message','original','genre']:
            
            if columns_perc[column] < 10:   
    
                # Selecting class slice
                feature = column
                mask = dataframe[feature] == 1
                df_temp = dataframe.loc[mask].copy()

                rows_target = int(dataframe.shape[0]*0.10) - df_temp.shape[0]

                obj_temp = df_temp.mean()
                obj_temp = obj_temp.sort_values(ascending=False)

                # Filtering data
                for key,value in obj_temp.iteritems():
                    if key not in ['id','message','original','genre']:
                        if (value*100) > 50:
                            if value * 100 == 100:
                                pass
                            else:
                                mask = df[key] == 0
                                df_temp = df_temp.loc[mask]

                df_temp.reset_index(inplace=True, drop=True)
                new_data = pd.DataFrame(columns = df_temp.columns)
                
                if df_temp.shape[0] > 0:

                    for _ in range(rows_target):
                        row_number = randint(0, df_temp.shape[0] - 1)

                        new_row = df_temp.iloc[row_number]
                        transformed_msg = synonym_tokenize(new_row['message'])
                        transformed_msg = ' '.join(transformed_msg)
                        new_row[0:]['message'] = transformed_msg
                        new_row[0:]['id'] = 'Artificial Data'
                        new_data = new_data.append(new_row)

                    print(f"{column} ADDED: {new_data.shape[0]}")

                    new_dataframe = new_dataframe.append(new_data)
    
    dataframe = dataframe.append(new_dataframe)
    
    return dataframe

#### 8.2.1 Oversampling with synonyms

In [24]:
df_new = oversample_nlp(df)
print("_"*70)
print(f"Default Dataframe: {df.shape[0]}")
print(f"New Dataframe: {df_new.shape[0]}")

offer ADDED: 2503
medical_help ADDED: 537
medical_products ADDED: 1308
search_and_rescue ADDED: 1897
security ADDED: 2150
military ADDED: 1761
water ADDED: 949
shelter ADDED: 307
clothing ADDED: 2216
money ADDED: 2017
missing_people ADDED: 2323
refugees ADDED: 1746
death ADDED: 1427
infrastructure_related ADDED: 916
transport ADDED: 1420
buildings ADDED: 1288
electricity ADDED: 2089
tools ADDED: 2462
hospitals ADDED: 2338
shops ADDED: 2501
aid_centers ADDED: 2312
other_infrastructure ADDED: 1470
floods ADDED: 466
storm ADDED: 178
fire ADDED: 2339
earthquake ADDED: 166
cold ADDED: 2091
other_weather ADDED: 1245
______________________________________________________________________
Default Dataframe: 26215
New Dataframe: 70637


#### 8.2.2. Dropping duplicates

In [25]:
# check number of duplicates
df_new.duplicated().sum()

6127

In [26]:
# Drop Duplicates
df_new.drop_duplicates(inplace=True)

#### 8.2.3 Splitting dataset

In [27]:
X_new = df_new['message']
Y_new = df_new.drop(columns=['id','message','original','genre'], axis=1)
for column in Y_new.columns:
    Y_new[column] = pd.to_numeric(Y_new[column])

In [28]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_new, Y_new, random_state=42)

#### 8.2.4 Creating model for the "new" dataset (RandomForest)

In [29]:
# Create pipeline with Classifier
moc = MultiOutputClassifier(RandomForestClassifier())

pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])

stop_words= stopwords.words('english')

##### Fitting model

In [30]:
%%time
pipeline2.fit(X_train2.values, y_train2.values)

CPU times: user 4min 38s, sys: 230 ms, total: 4min 38s
Wall time: 4min 39s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

#### Comparing Models

Original Dataset with AdaBoost X Improved Dataset with Random Forest

In [ ]:
new_y_pred = pipeline2.predict(X_test2)

In [32]:
show_model_dif(y_test, ada_y_pred, new_y_pred, y_test2)

       ACCURACY       | model 1: 0.9478 | model 2: 0.9736 -> dif:  0.0259
      PRECISION       | model 1: 0.5598 | model 2: 0.9308 -> dif:  0.3711
        RECALL        | model 1: 0.3229 | model 2: 0.7529 -> dif:    0.43
          F1          | model 1: 0.3945 | model 2: 0.8286 -> dif:  0.4341


#### 8.2.5. Tuning Model With New Dataset

In [33]:
%%time
cv = GridSearchCV(pipeline2, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(81)
new_tuned_model = cv.fit(X_train2.values, y_train2)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, score=0.8447584021079122, total= 2.9min
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, score=0.8391689022331978, total= 2.9min
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  6.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, score=0.8366932015989762, total= 2.9min
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=5 


...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 28.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=15, score=0.8254989645331886, total= 2.4min
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=15 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=15, score=0.8187358803362412, total= 2.4min
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=15 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=15, score=0.8318265448968103, total= 2.4min
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=100 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=100, score=0.7866689646490971, total= 2.2min
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=100 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=100, score=0.7828748926636417, total= 2.2min
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=100 


[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, score=0.4179901166558652, total= 1.4min
[CV] clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10 
[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, score=0.4142208889707708, total= 1.4min
[CV] clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10 
[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, score=0.4606865742210017, total= 1.4min
[CV] clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=15 
[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=15, score=0.38865462678042906, total= 1.4min
[CV] clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=15 
[CV]  clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=15, score=0.4435947970043358, total= 1.4min
[CV] clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_spli

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 144.5min finished


CPU times: user 2h 28min 32s, sys: 1.71 s, total: 2h 28min 34s
Wall time: 2h 29min 6s


#### Comparing Models

Improved Dataset with Random Forest X Improved Dataset with Tuned Random Forest

In [36]:
# Calculate evaluation metrics for test set
new_tuned_y_pred = new_tuned_model.predict(X_test2)

In [37]:
show_model_dif(y_test2, new_y_pred, new_tuned_y_pred , y_test2)

       ACCURACY       | model 1: 0.9736 | model 2: 0.974 -> dif:  0.0004
      PRECISION       | model 1: 0.9308 | model 2: 0.9348 -> dif:   0.004
        RECALL        | model 1: 0.7529 | model 2: 0.7529 -> dif:    -0.0
          F1          | model 1: 0.8286 | model 2: 0.8299 -> dif:  0.0013


#### 8.2.6 Creating model for the "new" dataset (AdaBoost)

In [38]:
# Create pipeline with Classifier
moc2 = MultiOutputClassifier(AdaBoostClassifier())

pipeline_ada2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc2)
    ])

In [39]:
%%time
pipeline_ada2.fit(X_train2.values, y_train2.values)

CPU times: user 6min, sys: 76 ms, total: 6min 1s
Wall time: 6min 1s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [40]:
new_ada_y_pred = pipeline_ada2.predict(X_test2)

#### Comparing Models

Improved Dataset with Tuned Random Forest X Improved Dataset with AdaBoost

In [41]:
show_model_dif(y_test2, new_tuned_y_pred, new_ada_y_pred, y_test2)

       ACCURACY       | model 1: 0.974 | model 2: 0.9475 -> dif: -0.0265
      PRECISION       | model 1: 0.9348 | model 2: 0.7979 -> dif: -0.1369
        RECALL        | model 1: 0.7529 | model 2: 0.5415 -> dif: -0.2115
          F1          | model 1: 0.8299 | model 2: 0.6347 -> dif: -0.1952


Improved Dataset with Random Forest X Improved Dataset with AdaBoost

In [42]:
show_model_dif(y_test2, new_y_pred, new_ada_y_pred, y_test2)

       ACCURACY       | model 1: 0.9736 | model 2: 0.9475 -> dif: -0.0261
      PRECISION       | model 1: 0.9308 | model 2: 0.7979 -> dif: -0.1329
        RECALL        | model 1: 0.7529 | model 2: 0.5415 -> dif: -0.2115
          F1          | model 1: 0.8286 | model 2: 0.6347 -> dif: -0.1939


### 9. Export your model as a pickle file

In [43]:
import pickle
with open('classifier.pkl', 'wb') as f:
    pickle.dump(new_tuned_model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.